Validation of the cutset conditioning algorithm: using the light model, compare my won implementation for 3 days against the variable elimination algorithm from the library. Both are exact inference algorithms, they should give the exact same results.

My won implementation: model with FEV1 noise, and temporal airway resistance

In [6]:
import itertools
import time

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import src.data.breathe_data as bd
import src.data.helpers as dh
import src.inference.helpers as ih
import src.modelling_ar.ar as ar
import src.inf_cutset_conditioning.helpers as cutseth
import src.models.builders as mb
from pgmpy.inference.ExactInference import VariableElimination

In [17]:
height = 180
age = 35
sex = "Male"
(
    model,
    inf_alg,
    HFEV1,
    HO2Sat,
    ecFEV1,
    AR,
    O2SatFFA,
    IA,
    UO2Sat,
    O2Sat,
    ecFEV1_2,
    AR_2,
    O2SatFFA_2,
    IA_2,
    UO2Sat_2,
    O2Sat_2,
) = mb.o2_sat_fev1_two_days_model_light(
    height,
    age,
    sex,
)

var_elim = VariableElimination(model)
res = var_elim.query(variables=[AR.name], evidence={ecFEV1.name: 2, HO2Sat.name: 1})
res.values

array([0.01178899, 0.03711222, 0.08648782, 0.25856762, 0.36499814,
       0.22242216, 0.01862306, 0.        , 0.        ])

In [18]:
height = 180
age = 35
sex = "Male"
mb.o2sat_fev1_fef2575_long_model_noise_shared_healthy_vars_and_temporal_ar_light(
    height,
    age,
    sex,
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tristan.trebaol/Desktop/PhD/Code/phd//src/models/cpts/AR_0_90_10_AR_0_90_10_DE_1_3_1.npy'